In [ ]:
# The MIT License (MIT)
# Copyright (c) 2020 Mehdi Ataei

# Import the Canvas class
from canvasapi import Canvas
import pandas as pd
# Canvas API URL (this link is for UofT API)
API_URL = "https://q.utoronto.ca/api/v1"
# Canvas API key (put yours here)
API_KEY = ""

canvas = Canvas(API_URL, API_KEY)

course_id = 139200
assignment_id = 321682

In [ ]:
# Get submissions in a dict with the student id as key
def get_all_submissions(users, course, assignment):
    submissions = {}
    for user in users:
        submissions[user.integration_id] = assignment.get_submission(
            user.id, include=['submission_comments'])
       
    return submissions  

In [ ]:
# Upload and put comment for one submission
def upload_grade_with_comment(submission, grade, ta_comment):
    
    comment = submission.submission_comments
    submission.edit(comment={'text_comment': ta_comment },
                    submission={'posted_grade': grade})

In [ ]:
# Read data (this part is specific to this assignment)
df = pd.read_excel('grades.xlsx')

In [ ]:
# Clean data (this part is specific to this assignment)
df = df.fillna('')
df['Student Number'] = df['Student Number'].astype(int)

In [ ]:
# Create comment dynamically (this part is specific to this assignment)
results = {}
for index, row in df.iterrows():
         
    student_comment = 'grading TA: ' + row.iloc[6] + ' '
    for i in range(8,30):
        student_comment += str(row.iloc[i]) + ' '
           
    results[row.iloc[1]] = {'comment': student_comment,
                            'grade': row['Total']}
    
print(results)

In [ ]:
# Request the course, users, and the assignment
course = canvas.get_course(course_id)
users = course.get_users(enrollment_type=['student'])
assignment = course.get_assignment(assignment_id)

In [ ]:
submissions = get_all_submissions(users, course, assignment)

In [ ]:
# Convert all keys (student ids) to int
submissions = {int(k):v for k,v in submissions.items()}

In [ ]:
# Post grades and comments automatically
for student_num, submission in submissions.items():
    try:

        grade = results[student_num]['grade']
        ta_comment = results[student_num]['comment']

        upload_grade_with_comment(submission, grade, ta_comment)

    except KeyError:
        
        print('error')